In [1]:
from random import randint, seed  # FIXME put import as, abych nebyla prase
# define seed to fix the random matrix
seed(0)

# returns matrix mu and list of squared euclidean norms
#   of the vectors in Gram-Schmidt orthogonalized basis
#   (Cholesky decomposition avoiding use of irrationals)
# G ... Gram matrix to be decomposed
def chol_decomp(G, cols=-1):
    rows = G.nrows()
    if cols < 0:
        cols = rows
    mu = matrix(QQ, rows, cols)
    squares = vector(QQ, cols)
    for y in range(rows):
        over_cols = (y>=cols)
        for x in range(cols if over_cols else y):
            t = 0
            for i in range(x):
                t += mu[y,i]*mu[x,i]*squares[i]
            mu[y,x] = (G[y,x]-t)/squares[x]
        if not over_cols:
            t = 0
            for i in range(y):
                t += mu[y,i]*mu[y,i]*squares[i]
            mu[y,y] = 1
            squares[y] = G[y,y]-t
    return (mu, squares)

# auxiliary function: swaps i-th row with the previous one
def swap_rows_in_mu(mu, B, i, n):
    mu[i-1], mu[i] = mu[i], mu[i-1]
    mu[i,i], mu[i-1,i] = 1, 0
    r = mu[i-1,i-1]
    mu[i-1,i-1] = 1
    temp = B[i-1]*r**2 + B[i]
    mu[i,i-1] = r*B[i-1]/temp
    B[i] = B[i-1]*B[i]/temp
    B[i-1] = temp
    for j in range(i+1, n):
        mu[j,i-1], mu[j,i] = mu[i,i-1]*mu[j,i-1]+mu[j,i]*(1-r*mu[i,i-1]), \
                           mu[j,i-1]-r*mu[j,i]

# auxiliary function: the inner cycle of LLL
def LLL_reduce_row(b, mu, i, h):
    for j in range(h-1, -1, -1):
        t = round(mu[i, j])
        for k in range(j+1):
            mu[i, k] -= t*mu[j, k]
        b[i] -= t*b[j]

# auxiliary function: LLL reduction algorithm
# b ... integer basis (does'n have to be square matrix)
# mu ... Gram-Schmidt mu matrix (doesn't have to be square matrix)
# B ... squared norms of Gram-Schmidt orthogonalized vectors
# n ... number of rows of b to be reduced (usually b.nrows())
# h ... index of first row not to be swapped (only reduced)
def LLL_aux(b, delta, mu, B, n, h):
    i = 1
    flag = True
    while i < h:
        if flag:
            LLL_reduce_row(b, mu, i, i)
        if i > 0 and (delta-mu[i,i-1]**2)*B[i-1] > B[i]:
            b[i], b[i-1] = b[i-1], b[i]
            swap_rows_in_mu(mu, B, i, n)
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    while i < n:  # performed only if n>h
        LLL_reduce_row(b, mu, i, h)
        i += 1
    return b, mu, B

# classical LLL algorithm
# last row of b can be used as a target point in Babai's algorithm
def LLL_own(b, delta=3/4, babai_bool=False):
    mu, B = chol_decomp(b*b.transpose(), b.nrows()-babai_bool)
    return LLL_aux(copy(b), delta, mu, B, b.nrows(), b.nrows()-babai_bool)[0]

# LLL for triangular input basis
def LLL_triangular_input(b, delta=3/4, babai_bool=False):
    B = vector(QQ, [b[j,j]**2 for j in range(b.ncols())])
    mu = matrix(QQ, b)
    for j in range(mu.ncols()):
        r = mu[j,j]
        for i in range(j, mu.nrows()):
            mu[i,j] /= r
    return LLL_aux(copy(b), delta, mu, B, b.nrows(), b.nrows()-babai_bool)[0]


# returns the length of SV
def shortestVectorLength(matrix):
    return matrix[0].norm()

# creates random square matrix of given dimension

def randomMatrix(dimension, perimeter):
    list = [randint(-perimeter, perimeter) for _ in range(dimension**2)]
    return matrix(ZZ, dimension, dimension, list)


In [32]:
def average_function(matrix, data):
    avg_1 = 0
    avg_2 = 0
    avg_3 = 0
    for data in data:
        #print(data[1],",", data[2])
        if data[1] == 1/2:
            avg_1 += data[2]
        if data[1] == 3/4:
            avg_2 += data[2]
        if data[1] == 99/100:
            avg_3 += data[2]
    avg_1 /= len(data)
    avg_2 /= len(data)
    avg_3 /= len(data)
    print(avg_1.n(), avg_2.n(), avg_3.n())


In [33]:
data = [] # matrix, delta, SVL

dimension = 10
perimeter = 50


for i in range(3):
    seed(i)
    B = randomMatrix(dimension, perimeter)    
    # print(B)
    for delta in [1/2, 3/4, 99/100]:
        BB = LLL_own(B, delta)
        data.append([B, delta, shortestVectorLength(BB)])
    average_function(BB, data)
    print("")






28.9578237365234 17.4260341634770 16.1176232049821

61.6550883180004 34.6015982007946 33.2931872422998

77.2185783579054 46.3772793558984 45.0688683974036

